Download ___doi___ data from ACM digital library.

01-03-2018

---

In [80]:
import time, random

In [121]:
import numpy as np

In [7]:
import requests

In [13]:
from bs4 import BeautifulSoup as Soup

In [8]:
agent = "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36"

In [9]:
base_url = 'https://dl.acm.org/citation.cfm'

---

#### Next link

Starting from a conference.

In [104]:
id_1 = '544220'

In [105]:
start_req = requests.get(base_url, params={'id': id_1, 'preflayout': 'flat'},
                         headers={'User-Agent': agent})

In [106]:
content = Soup(start_req.content, 'lxml')

The last `flatbody`

In [107]:
tab_div = content.find_all('div', class_='flatbody')[-1]

The 2nd div in `tab_div`

In [108]:
tmp_div = tab_div.find_all('div')[2]

The last hperlink (tag `a`) in the 1st div in `tmp_div`

In [109]:
a_tag = tmp_div.find('div').find_all('a')[-1]
a_tag

<a href="citation.cfm?id=827140&amp;picked=prox&amp;CFID=1023272311&amp;CFTOKEN=40704048" title="Next: JCDL '03">next proceeding <img align="absmiddle" alt="next" border="0" height="11" hspace="5" src="img/next.gif" width="19"/></a>

The link in the `a_tag`

In [110]:
next_id = a_tag['href'].split('&')[0].split('=')[-1]
next_id

'827140'

---

#### Get dois

In [63]:
table_tag = tab_div.find('table', class_='text12')

In [64]:
all_doi_a_tag = [a_ for span_tag in table_tag.find_all('span')\
                 for a_ in span_tag.find_all('a')]
len(all_doi_a_tag)

706

In [65]:
doi_list = list()

In [66]:
for a_ in all_doi_a_tag:
    try:
        if a_['title'] == 'DOI':
            doi_list.append(a_.text.strip())
    except:
        continue

In [67]:
len(doi_list)

116

---

#### Get all dois

In [111]:
def get_jcdl_dois(conf_id):
    start_req = requests.get(base_url, params={'id': conf_id, 'preflayout': 'flat'},
                             headers={'User-Agent': agent})
    content = Soup(start_req.content, 'lxml')
    # The last `flatbody`
    tab_div = content.find_all('div', class_='flatbody')[-1]
    # The 2nd div in `tab_div`
    tmp_div = tab_div.find_all('div')[2]
    # The last hperlink (tag `a`) in the 1st div in `tmp_div`
    a_tag = tmp_div.find('div').find_all('a')[-1]
    # The link in the `a_tag`
    next_id = a_tag['href'].split('&')[0].split('=')[-1]
    
    ## Get dois ##
    table_tag = tab_div.find('table', class_='text12')
    all_doi_a_tag = [a_ for span_tag in table_tag.find_all('span')\
                     for a_ in span_tag.find_all('a')]
    doi_list = list()
    for a_ in all_doi_a_tag:
        try:
            if a_['title'] == 'DOI':
                doi_list.append(a_.text.strip())
        except:
            continue
    return next_id, doi_list

##### Test

In [113]:
conf_id = '544220'

In [114]:
next_id, doi_list = get_jcdl_dois(conf_id)

In [115]:
next_id

'827140'

In [116]:
len(doi_list), doi_list[:4]

(135,
 ['10.1145/544220.544222',
  '10.1145/544220.544223',
  '10.1145/544220.544224',
  '10.1145/544220.544226'])

##### Apply

In [117]:
next_id = '379437'
doi_list = list()

In [118]:
for i in range(15):
    next_id, doi = get_jcdl_dois(next_id)
    doi_list.extend(doi)
    print(i, next_id, len(doi_list))
    print('--------------')
    time.sleep(random.randint(2, 5) + random.random())

0 544220 116
--------------
1 827140 251
--------------
2 996350 251
--------------
3 1065385 368
--------------
4 1141753 496
--------------
5 1255175 604
--------------
6 1378889 721
--------------
7 1555400 833
--------------
8 1816123 945
--------------
9 1998076 1011
--------------
10 2232817 1113
--------------
11 2467696 1212
--------------
12 2740769 1309
--------------
13 2756406 1309
--------------
14 2910896 1376
--------------


In [119]:
len(doi_list)

1376

In [120]:
len(set(doi_list))

1376

##### Save doi list

In [122]:
np.savetxt(X=doi_list, fname='JCDL-DOI-01-16.csv', fmt='%s')